In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\jimco\anaconda3\envs\pythondata\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [42]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import os
#import keras

In [18]:
# fix random seed for reproducibility
seed=231
np.random.seed(231)

# Read the CSV and Perform Basic Data Cleaning

In [19]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [20]:
# Set features. This will also be used as your x values.
# selected_features = df[['names', 'of', 'selected', 'features', 'here']]

# 'koi_disposition',
# 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 
# 'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2', 
# 'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact', 
# 'koi_impact_err1', 'koi_impact_err2', 'koi_duration', 'koi_duration_err1', 
# 'koi_duration_err2', 'koi_depth', 'koi_depth_err1', 'koi_depth_err2', 'koi_prad', 
# 'koi_prad_err1', 'koi_prad_err2', 'koi_teq', 'koi_insol', 'koi_insol_err1', 
# 'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1', 
# 'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad', 
# 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec'

selected_features = df[['koi_disposition',
                        'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 
                        'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2', 
                        'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact', 
                        'koi_impact_err1', 'koi_impact_err2', 'koi_duration', 'koi_duration_err1', 
                        'koi_duration_err2', 'koi_depth', 'koi_depth_err1', 'koi_depth_err2', 
                        'koi_prad', 'koi_prad_err1', 'koi_prad_err2', 'koi_teq', 'koi_insol', 
                        'koi_insol_err1', 'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num', 
                        'koi_steff', 'koi_steff_err1', 'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 
                        'koi_slogg_err2', 'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec' ]]

selected_features.shape


(6991, 40)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [23]:
data = selected_features.values
X = data[:, 1:40]
y = data[:, 0]

y = y.reshape(-1,1)

print(X.shape)
print(y.shape)

(6991, 39)
(6991, 1)


In [26]:
# Split the training and test data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)

In [27]:
X_train[:2]

array([[0, 1, 0, 0, 14.79721916, 4.11e-07, -4.11e-07, 144.6959608,
        2.27e-05, -2.27e-05, 0.157, 0.01, 0.0, 7.61037, 0.00059,
        -0.00059, 252700.0, 36.4, -36.4, 47.71, 14.04, -4.66, 787, 90.8,
        78.38, -26.16, 7795.3, 1, 6343, 171, -209, 4.495,
        0.052000000000000005, -0.20800000000000002, 0.9309999999999999,
        0.27399999999999997, -0.091, 292.02866, 42.669552],
       [0, 0, 0, 0, 5.95958637, 4.4e-05, -4.4e-05, 135.00822, 0.00449,
        -0.00449, 0.26, 0.198, -0.259, 2.8989999999999996, 0.15, -0.15,
        133.1, 9.6, -9.6, 1.02, 0.25, -0.08, 897, 153.48, 111.26, -38.24,
        16.1, 1, 5452, 164, -164, 4.535, 0.03, -0.17, 0.9, 0.214, -0.071,
        286.92429, 47.053740999999995]], dtype=object)

In [28]:
y_train[:5]

array([['FALSE POSITIVE'],
       ['CANDIDATE'],
       ['FALSE POSITIVE'],
       ['FALSE POSITIVE'],
       ['CONFIRMED']], dtype=object)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [34]:
# set up scaler for the X data
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)

In [35]:
# Remember to scale both the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [41]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)

(5243, 39)
(1748, 39)


### One Hot Encode the y values

In [36]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y_test = label_encoder.transform(y_test)
encoded_y_train = label_encoder.transform(y_train)

C:\Users\jimco\Anaconda3\envs\PythonData\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jimco\Anaconda3\envs\PythonData\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [37]:
for label, original_class in zip(encoded_y_test, y_test):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Origi

Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
--------

Encoded Label: 0
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITI

Encoded Label: 1
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CONFIRM

Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original C

Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class:

------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CANDIDATE']
Encoded Label: 0
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['FALSE POSITIVE']
Encoded Label: 2
------------
Original Class: ['CONFIRMED']
Encoded Label: 1
------------
Original Class: ['CANDIDATE']
Encoded Labe

In [55]:
# #### step 2 - One hot encode the label encoded data
# # One-hot encoding label encoded data
# from tensorflow.keras.utils import to_categorical

# y_train_categorical = to_categorical(encoded_y_train)
# y_test_categorical = to_categorical(encoded_y_test)
# y_train_categorical.shape

# Train the Model



In [56]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train_scaled, encoded_y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [47]:
print('Test Acc: %.3f' % model.score(X_test_scaled, encoded_y_test))

Test Acc: 0.883


## Making Predictions with new data

We can use our trained model to make predictions using `model.predict`

In [48]:
import numpy as np
test = np.expand_dims(X_test[1], axis=0)
test.shape

(1, 39)

In [49]:
test = np.asarray(test).astype(np.float32)

In [97]:
prediction = model.predict(test)
print(f"Predicted class: {prediction}, actual class: [{encoded_y_test[1]}]")

Predicted class: [1], actual class: [1]


In [52]:
label_encoder.inverse_transform(prediction)

array(['CONFIRMED'], dtype=object)

In [54]:
print(f"Training Data Score: {model.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 0.8916650772458516
Testing Data Score: 0.8827231121281465


In [87]:
# Make predictions with the hypertuned model
predictions = model.predict(X_test_scaled)

In [88]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(encoded_y_test, predictions,
                            target_names=["CANDIDATE", "CONFIRMED", "FALSE POSITIVE"]))

                precision    recall  f1-score   support

     CANDIDATE       0.82      0.70      0.76       427
     CONFIRMED       0.76      0.84      0.80       458
FALSE POSITIVE       0.98      1.00      0.99       863

      accuracy                           0.88      1748
     macro avg       0.85      0.85      0.85      1748
  weighted avg       0.88      0.88      0.88      1748



# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [77]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01],
              'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
              'degree' : [2,3,5,7]
             }

# n_jobs = -1 uses all cores of the computer and parallizes the gridsearch
# side effect is that most of the output goes to console window instead of here in jupyter notebook.
grid = GridSearchCV(model, param_grid, verbose=3, n_jobs=-1)

In [78]:
# Train the model with GridSearch
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train_scaled, encoded_y_train)

C:\Users\jimco\Anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   42.4s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:  2.5min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='linear',
                           max_iter=-1, probability=False, random_state=None,
                           shrinking=True, tol=0.001, verbose=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [1, 5, 10], 'degree': [2, 3, 5, 7],
                         'gamma': [0.0001, 0.001, 0.01],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [79]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 5, 'degree': 2, 'gamma': 0.0001, 'kernel': 'linear'}
0.890520694259012


In [80]:
## Fit Model using best params
kernel = grid.best_params_['kernel']
C = grid.best_params_['C']
gamma = grid.best_params_['gamma']
degree = grid.best_params_['degree']

print(kernel, C, gamma, degree)

linear 5 0.0001 2


In [81]:
best_model = SVC(kernel=kernel, C=C, gamma=gamma, degree=degree)
best_model.fit(X_train_scaled, encoded_y_train)

SVC(C=5, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=2, gamma=0.0001, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [82]:
print(f"Training Data Score: {model.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 0.8916650772458516
Testing Data Score: 0.8827231121281465


In [89]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)

In [90]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(encoded_y_test, predictions,
                            target_names=["CANDIDATE", "CONFIRMED", "FALSE POSITIVE"]))

                precision    recall  f1-score   support

     CANDIDATE       0.82      0.71      0.76       427
     CONFIRMED       0.76      0.84      0.80       458
FALSE POSITIVE       0.98      1.00      0.99       863

      accuracy                           0.88      1748
     macro avg       0.85      0.85      0.85      1748
  weighted avg       0.89      0.88      0.88      1748



# Save the Model

In [84]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'best_SVC_model.sav'
joblib.dump(best_model, filename)

['best_SVC_model.sav']

In [ ]:
# load model and test

In [85]:
loaded_model = joblib.load(filename)

In [101]:
#y_test

In [99]:
prediction = loaded_model.predict(test)
print(f"Predicted class: {prediction}, actual class: [{encoded_y_test[1]}]")

Predicted class: [1], actual class: [1]


In [100]:
label_encoder.inverse_transform(prediction)

array(['CONFIRMED'], dtype=object)

In [102]:
# Make predictions with the hypertuned model
predictions = loaded_model.predict(X_test_scaled)

In [103]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(encoded_y_test, predictions,
                            target_names=["CANDIDATE", "CONFIRMED", "FALSE POSITIVE"]))

                precision    recall  f1-score   support

     CANDIDATE       0.82      0.71      0.76       427
     CONFIRMED       0.76      0.84      0.80       458
FALSE POSITIVE       0.98      1.00      0.99       863

      accuracy                           0.88      1748
     macro avg       0.85      0.85      0.85      1748
  weighted avg       0.89      0.88      0.88      1748



In [104]:
ls

 Volume in drive C is Local Disk
 Volume Serial Number is 1A76-E077

 Directory of C:\Users\jimco\Desktop\DABC_GIT\DABC_hw\machine-learning-challenge

03/24/2020  09:43 PM    <DIR>          .
03/24/2020  09:43 PM    <DIR>          ..
03/24/2020  08:10 PM    <DIR>          .ipynb_checkpoints
03/23/2020  12:42 AM           334,296 3hidden_decentmodel.h5
03/24/2020  09:25 PM           573,824 best_SVC_model.sav
03/23/2020  11:07 AM         1,804,949 exoplanet_data.csv
03/21/2020  10:26 AM             1,166 firstModel.sav
03/21/2020  10:46 AM            77,296 firstModel_trained.h5
03/23/2020  11:46 PM             1,621 grid_best_estimator
03/20/2020  11:34 PM    <DIR>          Homework
03/23/2020  11:08 AM            12,288 koi.docx
03/24/2020  09:42 PM           315,802 model_1.ipynb
03/24/2020  09:38 PM           201,105 model_SVC.ipynb
03/24/2020  09:43 PM           276,875 NN_grid_best_estimator.pkl
03/20/2020  09:12 PM                29 README.md
              11 File(s)      3,599,2